#Please replace the **YYYYYY** with the **absolute path** of the install directory of OpenPose

In [0]:
!rm -r ./PartX/tmp_json/
!mkdir ./PartX/tmp_json/
!YYYYYY/openpose/build/examples/openpose/openpose.bin --video ./PartX/video.mp4 --write_json ./PartX/tmp_json --render_pose 0 --number_people_max 1 --keypoint_scale 3 --display 0 --face

#Install Packages

In [0]:
!pip install tensorflow==1.15.0
!pip install Keras

In [0]:
!pip install opencv-python
!pip install matplotlib

#Read input video

In [0]:
import json
import pandas as pd
import numpy as np
import os, os.path
import csv
import cv2

In [0]:
N_FACE = 70
N_HAND = 21
N_FEATURES = (N_FACE + N_HAND * 2) * 2
MODEL_FOLDER = './model/'
VIDEO_FOLDER = './PartX/'
JSON_FOLDER = './PartX/tmp_json/'
CSV_FOLDER = './PartX/'
FOLDER = './'
NAME = '827008769'

N_FACE = 70
N_POSE = 25
N_HAND = 21
COL_NAMES = ['frame']
COL_NAMES += ['f_' + str(i) for i in range(N_FACE * 2)]
COL_NAMES += ['p_' + str(i) for i in range(N_POSE * 2)]
COL_NAMES += ['lh_' + str(i) for i in range(N_HAND * 2)]
COL_NAMES += ['rh_' + str(i) for i in range(N_HAND * 2)]

droplist_f = set([(i * 3 + 2) for i in range(N_FACE)])
droplist_p = set([(i * 3 + 2) for i in range(N_POSE)])
droplist_h = set([(i * 3 + 2) for i in range(N_HAND)])
idx_f = [i for i in range(N_FACE * 3) if (i not in droplist_f)]
idx_p = [i for i in range(N_POSE * 3) if (i not in droplist_p)]
idx_h = [i for i in range(N_HAND * 3) if (i not in droplist_h)]

In [0]:
# collect the json output of openpose and combine them into csv
def json_to_csv(folderpath, csv_path, csv_writer):
  for name in sorted(os.listdir(folderpath)): 
    if name[-4:] == 'json':
      filepath = os.path.join(folderpath, name)   
      if os.path.isfile(filepath):
        with open(filepath, 'r') as f:
          data = json.load(f)['people']
          
          if len(data) > 0:
            row = [int(name[12:-15])]
            row_face = [float(data[0]['face_keypoints_2d'][j]) for j in idx_f]
            row_pose = [float(data[0]['pose_keypoints_2d'][j]) for j in idx_p]
            row_lh = [float(data[0]['hand_left_keypoints_2d'][j]) for j in idx_h]
            row_rh = [float(data[0]['hand_right_keypoints_2d'][j]) for j in idx_h]           
          else:
            row = [int(name[12:-15])]
            row_face = [float(0) for j in idx_f]
            row_pose = [float(0) for j in idx_p]
            row_lh = [float(0) for j in idx_h]
            row_rh = [float(0) for j in idx_h]
          
          row += row_face + row_pose + row_lh + row_rh            
          row = np.array(row)
          csv_writer.writerow(row)
          cnt += 1

csv_path = CSV_FOLDER + 'video.csv'
write_file = open(csv_path, 'w')
csv_writer = csv.writer(write_file) 
csv_writer.writerow(COL_NAMES)
json_to_csv(JSON_FOLDER, csv_path, csv_writer)

#Functions

##Normalize

In [0]:
# normalize
def normalize(df):
  x_df = df.iloc[:,2:].copy()
  num_df = x_df.sub(x_df[x_df > 0].min(axis=1) - 0.0001, axis=0)
  den_df = x_df.max(axis=1) - (x_df[x_df > 0.1].min(axis=1) - 0.0001)
  x_df = num_df.div(den_df, axis=0)
  x_df[x_df < 0] = 0
  df.iloc[:,2:] = x_df

##Reshape Data into Segments and Prepare for Keras

In [0]:
# slice every TIME_STEP frame as a segment until from 1st frame to last, if null, make up with 0
def get_slices(df, TIME_STEP, cols, fps):
  slices = []
  ts = []
  df = df[cols]
  for i in range(df.shape[0]):
    ts.append(i/fps)
    if i + 1 < TIME_STEP:
      continue
    else:
      slices.append(df.values[df.index[i + 1 - TIME_STEP : i + 1]])

  slices_arr = np.asarray(slices, dtype= np.float32)
  ts_arr = np.asarray(ts[TIME_STEP - 1:], dtype= np.float32)
  return slices_arr, ts_arr

##Draw figure

In [0]:
import matplotlib.pyplot as plt
def drawplt(time, pred, name):
  plt.figure(figsize=(10,8))
  plt.plot(time, pred, 'b', label='prediction', markersize=3, color='red')
  plt.title('Prediction over time')
  my_y_ticks = np.arange(0, 1.1, 0.1)
  plt.yticks(my_y_ticks)
  plt.legend()
  plt.savefig(name)

##Generate json

In [0]:
def to_json(name, ts_arr, y_pred):
  np_pred = np.array((ts_arr, y_pred.flatten())).T
  pred_list = np_pred.tolist()
  pred_dict = {"cry":pred_list}
  pred_json = json.dumps(pred_dict)
  with open(name, 'w') as json_file:
    json_file.write(pred_json)

#Prediction

In [0]:
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, LSTM, Dropout, Flatten, Conv1D, Activation, MaxPooling1D, Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import History, EarlyStopping, ModelCheckpoint
from keras.models import load_model

TensorFlow 1.x selected.


Using TensorFlow backend.


In [0]:
def get_pred(name, model, time_step):
  X = pd.read_csv(CSV_FOLDER + name + '.csv')
  normalize(X)
  # Get fps
  videoCapture = cv2.VideoCapture(VIDEO_FOLDER + name + '.mp4')
  fps = videoCapture.get(cv2.CAP_PROP_FPS)
  # get input and time array
  cols = ['f_' + str(i) for i in range(N_FACE * 2)]
  cols += ['lh_' + str(i) for i in range(N_HAND * 2)]
  cols += ['rh_' + str(i) for i in range(N_HAND * 2)]
  X_arr, ts_arr = get_slices(X, time_step, cols, fps)
  print(X_arr.shape, ts_arr.shape)
  # get prediction
  y_pred = model.predict(X_arr)
  # draw plot
  fig_name = FOLDER + NAME + '.png'
  drawplt(ts_arr, y_pred, fig_name)
  # output json file
  js_name = FOLDER + NAME + '.json'
  to_json(js_name, ts_arr, y_pred)

In [0]:
model = load_model(MODEL_FOLDER + 'model1_15_8247.h5')
get_pred('video', model, 15)